In [4]:
import pandas as pd
import psycopg2
import psycopg2.extras
from psycopg2.extras import RealDictCursor


connection = psycopg2.connect(
    database = 'volebnikalkulacka',
    user = 'postgres',
    password = 'root',
    )

In [92]:
SQL_Query = pd.read_sql_query(
'''
SELECT zkratka,vysledek
FROM psp_data_hl_poslanec AS hp 
	INNER JOIN psp_data_poslanec AS p 
	ON hp.id_poslanec = p.id_poslanec 
    
    INNER JOIN psp_data_zarazeni AS z
    ON p.id_osoba = z.id_osoba
    
    INNER JOIN psp_data_organy as o
    ON z.id_of = o.id_organ
    
    INNER JOIN psp_data_osoby as os
    ON os.id_osoba = p.id_osoba
    
WHERE hp.id_hlasovani = 67306 --konkretni jedno hlasovani
AND z.cl_funkce = 0 --clenstvi
AND z.do_o IS NULL
AND o.organ_id_organ = 172 --Aktualni volebni obdobi
AND o.id_typ_organu = 1 --Klub


''', connection)

df = pd.DataFrame(SQL_Query)

In [82]:
df.groupby(['zkratka', 'vysledek']).tail()

,zkratka,vysledek
27,STAN,B
30,TOP09,B
33,SPD,B
35,SPD,A
37,ČSSD,A
38,KDU-ČSL,B
41,ČSSD,A
47,KDU-ČSL,B
52,KDU-ČSL,B
53,Nezařaz,B


In [93]:
df['zkratka']

0          ODS
1          ANO
2         KSČM
3          ANO
4          ANO
        ...   
180        ODS
181       KSČM
182     Piráti
183    KDU-ČSL
184        ANO
Name: zkratka, Length: 185, dtype: object

In [127]:
total_difference = 0
for party in df['zkratka'].unique():
    pro_df = df.query(f'zkratka=="{party}" & vysledek == "A"')
    proti_df = df.query(f'zkratka=="{party}" & (vysledek == "B" | vysledek == "N" )').count()
    members_count = len(df.query(f'zkratka=="{party}" & (vysledek == "A" | vysledek == "B" | vysledek == "N" ) ').index)
    
    if(members_count == 0):
        break
    
    pro_percentage = len(pro_df.index)/members_count
    proti_percentage = len(proti_df.index)/members_count
    party_difference = 1 - abs(pro_percentage-proti_percentage)
    
    total_difference += party_difference * (members_count/185) * 100
    
total_difference

24.864864864864863

In [52]:
df_filtered = df.query('zkratka=="ANO" & vysledek == "A"')
df_filtered['count'].count()

1